# PATH

구글 드라이브

In [1]:
import os
import sys
from google.colab import drive

drive.mount('/content/drive')
# 지금 RNN이란 폴더에서 작업해서 RNN으로 맞춘 상태입니다! 
os.chdir('/content/drive/MyDrive/Colab Notebooks/RNN 과제')

Mounted at /content/drive


In [2]:
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/RNN 과제'

In [3]:
root_data = f"{os.getcwd()}/data/data_hw"
root_data

'/content/drive/MyDrive/Colab Notebooks/RNN 과제/data/data_hw'

In [4]:
# 데이터 불러올 때 사용할 경로로 이후에 있을 unzip과 연동되는 내용입니다! 
path_train_data = "data/data_hw/1.Training/원천데이터/일상생활및구어체_영한_train_set.json"

path_val_data = "data/data_hw/2.Validation/원천데이터/일상생활및구어체_영한_valid_set.json"


# 사전 준비 사항

In [5]:
import torch

In [6]:
device = 'cuda' if torch.cuda.is_available() else "cpu"

In [7]:
device

'cuda'

# Data

실습에서 사용했던 영어-프랑스어와 달리 현재 사용할 데이터 셋은 영어-한국어 데이터 셋입니다. 

특히 압축을 풀어야 사용하실 수 있으니 유의하여 진행해주시길 바랍니다.

다행인 점은 용량이 그리 크지 않을겁니다!

저장 경로는 위에서 path_train_data와 path_val_data와 동일하게 맞췄습니다.

In [8]:
#!unzip -qq data/data_hw/1.Training/원천데이터/TS1.zip -d data/data_hw/1.Training/원천데이터

#!unzip -qq data/data_hw/2.Validation/원천데이터/VS1.zip -d data/data_hw/2.Validation/원천데이터


## 데이터 구조 확인

In [9]:
import json

In [10]:
# 경로는 앞서 PATH에서 선언했고 unzip에서도 경로를 지정했습니다. 만약 여기에 변화가 생기신다면 에러가 생길 수 있으니 유의하셔야 합니다.
with open(path_train_data, 'r') as f:
  raw_train_data = json.load(f)

with open(path_val_data, 'r') as f:
  raw_val_data = json.load(f)


In [11]:
# 현재 데이터의 구조가 {'data':~~~ }로 되어있는것을 알 수 있습니다.
raw_train_data.keys()

dict_keys(['data'])

In [12]:
# 학습 데이터 개수 확인
len(raw_train_data['data'])

1200307

지금 데이터가 너무 많아서 전처리만 해도 1,2시간 걸릴 수 있습니다. 그래서 숙제에서는 매우 일부 데이터만 이용하도록 하겠습니다.

In [13]:
raw_train_data = raw_train_data['data'][:10000]
raw_val_data = raw_val_data['data'][:1000]

In [14]:
# 샘플로 하나 확인
sample_data = raw_train_data[0]

In [15]:
# 아래의 예시를 보고 데이터를 어떻게 처리할지 잘 생각하셔야 합니다!
# 보았을 때 우리가 사용해야 하는 건 ko와 en가 되고
# 실제 데이터를 사용한다고 할 때 이 외에 필요한 데이터가 더 있을 수 있습니다.
# 모델링 과정에서는 어떻게 데이터들이 저장되어 있는지 주의해서 처리하시면 됩니다.
sample_data

{'sn': 'ECOAR1A00003',
 'data_set': '일상생활및구어체',
 'domain': '해외고객과의채팅',
 'subdomain': '숙박,음식점',
 'en_original': "I'm glad to hear that, and I hope you do consider doing business with us.",
 'en': "I'm glad to hear that, and I hope you do consider doing business with us.",
 'mt': '그 소식을 들으니 기쁩니다. 우리와 거래하는 것을 고려해 보시기 바랍니다.',
 'ko': '그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다.',
 'source_language': 'en',
 'target_language': 'ko',
 'word_count_ko': 10.0,
 'word_count_en': 15.0,
 'word_ratio': 0.667,
 'file_name': '해외고객과의채팅_숙박,음식점.xlsx',
 'source': '크라우드 소싱',
 'license': 'open',
 'style': '구어체',
 'included_unknown_words': False,
 'ner': None}

# Preprocess

세션 설명 코드에서는 알파벳 하나하나 사용했지만, 이번 과정에서는 토큰화를 진행하여 사용하는 과정을 Dataset에 바로 적용해보고자 합니다! 

토큰화는 앞서 word embedding 세션에서 사용한 것들을 그대로 이용하고자 합니다. 세부 커스터마이징은 자유롭습니다.

전처리 과정에서 중요한 점은, 앞의 구현 코드에서 여러 개의 for loop을 이용해서 구현했었지만, 실제 데이터를 사용한다면 매우 오래 걸리게 됩니다. 실제 데이터를 사용할 때엔 결국 얼마나 효율적으로 빠르게 정리할 수 있을지 고민해보시면서 접근하시길 바랍니다.

즉, 전처리에서 챙길 통찰은 얼마나 "효율적"으로 할 수 있는가? 입니다.

## Tokenizer

In [16]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http:

In [17]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [18]:
from konlpy.tag import Okt

In [19]:
okt = Okt()

In [20]:
def CustomTokenizer(corpus):
    stop = ['Josa','Punctuation', 'Foreign', 'Number']
    tokenized = []
    for i, j in okt.pos(corpus, stem = True, norm = True):
      if (j in stop) and (i not in ['!', '?']):
        continue
      else:
        tokenized.append(i) 

    return tokenized      

In [21]:
# 한국어 처리 어떻게 되고 있는지 확인
CustomTokenizer(sample_data['ko'])

['그', '말', '들다', '기쁘다', '저희', '거래', '하다', '것', '고려', '하다', '주다', '하다']

In [22]:
# 영어도 일단 토큰화가 되고 있으니 사용해보도록 합시다
CustomTokenizer(sample_data['en'])

['I',
 'm',
 'glad',
 'to',
 'hear',
 'that',
 'and',
 'I',
 'hope',
 'you',
 'do',
 'consider',
 'doing',
 'business',
 'with',
 'us']

## 본격적인 전처리 과정

전처리릏 할 때 주의할 점은, 우리가 가지고 있는 것은 train 데이터 뿐이라는 점입니다.

토큰화를 하고, token들을 수집할 때 평가데이터(혹은 테스트 데이터)에 대한 정보는 가지고 있을 수 없습니다. 그렇기 때문에 train data를 기준으로 전처리가 진행되어야 합니다.

### Tokenization

In [23]:
from tqdm import tqdm

In [24]:
src_vocab = set()
tar_vocab = set()

src_seq_tr = []
tar_seq_tr = []

# 전체 raw data에서 문장하나씩 loop 돌아가는 것
for raw_tr_dat in tqdm(raw_train_data):
  # 문장별 tokenization
  src_tmp = CustomTokenizer(raw_tr_dat['ko'])
  tar_tmp = CustomTokenizer(raw_tr_dat['en'])

  # 입력 시퀀스에서 문장의 끝을 알리는 <EOS> 토큰 추가
  # 출력 시퀀스에서 시작과 끝을 알리는 <SOS>, <EOS> 토큰 추가
  src_tmp.append('<EOS>')
  tar_tmp.insert(0, '<SOS>')
  tar_tmp.append('<EOS>')

  # 한국어, 영어 단어집합 구성
  src_vocab.update(src_tmp)
  tar_vocab.update(tar_tmp)

  # tokenization 끝난 문장 보관
  src_seq_tr.append(src_tmp)
  tar_seq_tr.append(tar_tmp)

100%|██████████| 10000/10000 [00:33<00:00, 298.72it/s]


In [25]:
src_vocab

{'불행하다',
 '오늘이',
 '육성',
 '흥미',
 '요크셔',
 '우주',
 '주선',
 '훑다',
 '크리에이티브',
 '웹디자인',
 '개시',
 '대접',
 '묵시',
 '급등',
 '종로',
 '컴프레서',
 '신호',
 '인증서',
 '태블릿',
 '팀',
 '표면',
 '상의',
 'PDA',
 '견인차',
 '속',
 '쇼룸',
 '자식',
 '단순하다',
 '만드나',
 '궁궐',
 '모범',
 '디스크',
 '드라이브',
 '깻잎',
 '침체',
 '서나',
 '경',
 '앞',
 '내려가다',
 '거점',
 '여러',
 '참가자',
 '유머',
 '디스플레이',
 '공정',
 '렌즈',
 '노래',
 '메타',
 '뒷받침',
 '핫스팟',
 '어긋나다',
 '유연성',
 '급하다',
 '정규',
 '습니',
 '환자',
 '하와이',
 '중고',
 '워크숍',
 '회차',
 '흙빛',
 '표준',
 '동의',
 '용이하다',
 '개다',
 '조심하다',
 '풍',
 '블렌드',
 'Private',
 '유리하다',
 '구매',
 '까다롭다',
 '편',
 '치르다',
 '축하',
 '특급',
 '마우스',
 '살이',
 '이유',
 '일련',
 '자연',
 '파티',
 '사본',
 '각본',
 '우산',
 '강제',
 '평점',
 '조도',
 '아침식사',
 '발급',
 '부탁',
 '출장',
 '매수',
 '주어지다',
 '더하다',
 '분당',
 '정리',
 '컬래버레이션',
 '시험관',
 '콘솔',
 'Food',
 '월요일',
 '양도',
 '분류',
 '존경',
 '우린',
 '심기',
 '춤',
 '제품',
 '그랬더니',
 '침',
 '불량',
 '예술',
 '몇몇',
 '웜',
 '라이',
 '플래닝',
 '사태',
 '쿠션',
 '남편',
 '산출',
 '고구마',
 '타겟',
 '고개',
 '가스',
 '강남역',
 '빌트',
 '식단',
 '포스터',
 '담당',
 '세척',
 '문의',
 '협조',
 '알아듣다'

In [26]:
src_tmp

['안녕하다',
 '저희',
 '한국',
 '있다',
 '저희',
 '회사',
 '새',
 '사무실',
 '위',
 '인테리어',
 '디자이너',
 '찾다',
 '있다',
 '<EOS>']

고민해볼 점: 과연 한국어 단어집합에 한국어만 있을까요?

지금 단계에서는 시간 상 체크하진 않지만 고민해볼 부분이기도 합니다.

### token - index 정리

In [27]:
# 최종 정수 인코딩 하기 전 보기 편하기 위해 sort
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

In [28]:
# token - idx dictionary
# 여기서도 동일하게 i+1로 해줘야 padding에 사용할 token을 0으로 사용할 수 있습니다!
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{'!': 1, '<EOS>': 2, '?': 3, 'A': 4, 'AAA': 5, 'AAA1@BBB1.com': 6, 'AAA@BBB.com': 7, 'ABCDE': 8, 'AI': 9, 'ANR': 10, 'ATM': 11, 'Associates': 12, 'B': 13, 'BBB': 14, 'BBB@email.com': 15, 'BSE': 16, 'Bank': 17, 'C': 18, 'CCTV': 19, 'CEO': 20, 'CFM': 21, 'CMS': 22, 'COVID': 23, 'CSS': 24, 'CT': 25, 'Company': 26, 'Corporation': 27, 'D': 28, 'DCA': 29, 'DVR': 30, 'Drive': 31, 'E': 32, 'EOB': 33, 'EPP': 34, 'ESG': 35, 'ETF': 36, 'EVA': 37, 'Enter': 38, 'Expo': 39, 'FFF': 40, 'FFFF': 41, 'FOMO': 42, 'Fair': 43, 'Fi': 44, 'Food': 45, 'Fu': 46, 'G': 47, 'GB': 48, 'GPS': 49, 'GVWR': 50, 'HDD': 51, 'HTML': 52, 'ID': 53, 'IP': 54, 'IPO': 55, 'IT': 56, 'IVR': 57, 'IaaS': 58, 'International': 59, 'IoT': 60, 'JLPT': 61, 'K': 62, 'Kabsat': 63, 'Korea': 64, 'LED': 65, 'Limited': 66, 'Locust': 67, 'M': 68, 'MAR': 69, 'MMS': 70, 'MOU': 71, 'Medical': 72, 'NFT': 73, 'NI': 74, 'NSE': 75, 'NTFS': 76, 'PDA': 77, 'PDF': 78, 'PPE': 79, 'PRO': 80, 'PSA': 81, 'PVP': 82, 'Precision': 83, 'Private': 84, 'QR': 85

### 정수 인코딩

In [29]:
# 앞서 toekn-index dictionary로 정리한 것을 이용해서 각각의 토큰들을 정수인코딩을 진행해봅니다.
# 이 과정에서 유의할 점은, 기존의 구현 코드에서 여러개의 for loop을 사용했습니다.
# 이보다 빨리 진행하기 위해 어떻게 코딩할 수 있을까요?
# 시간이 충분하시다면 for loop을 그대로 이용하셔도 좋습니다만, 다른 방법도 고민하셔도 좋습니다.
# hint: Pandas에서 apply와 비슷한 것들을 이용해보면 어떨까요?

# encoder의 입력값 정수 인코딩 진행
encoder_input = list(map(lambda words: torch.Tensor(list(map(lambda c: src_to_index[c], words))).long(), src_seq_tr))
# 우선 decoder에서 사용하는 문장에 대해 정수인코딩 진행
# decoder의 입력값은 마지막 <EOS>가 들어가면 안되니 마지막 토큰은 빼주기
decoder_input = list(map(lambda words: torch.Tensor(list(map(lambda c: tar_to_index[c], words))[:-1]).long(), tar_seq_tr))
# 다만 decoder의 출력값은 <SOS>들어가면 안되니 첫번째 토큰은 빼주기
decoder_target = list(map(lambda words: torch.Tensor(list(map(lambda c: tar_to_index[c], words))[1:]).long(), tar_seq_tr))

In [30]:
src_seq_tr[1]

['확실하다', '생각', '있다', '몇', '가지', '여쭈다', '보고', '싶다', '게', '있다', '<EOS>']

In [31]:
from torch.nn.utils.rnn import pad_sequence

### Padding

In [32]:
# 현재 길이가 제각각인 정수화된 문장이 있습니다. padding을 통해 길이를 맞춰봅시다
# 주의사항: 여기에 사용된 값을은 정수일까요?
batch_first = True

encoder_input_tr =  pad_sequence(encoder_input, batch_first=batch_first).long()
decoder_input_tr = pad_sequence(decoder_input, batch_first=batch_first).long()
decoder_target_tr = pad_sequence(decoder_target, batch_first=batch_first).long()

## Train data

tuple type으로,  (encoder_input, decoder_input, decoder_target)의 순서를 가지게 됩니다. 이는 tuple assignment를 이용하려고 하는 것이니 자유롭게 사용하셔도 좋습니다.

In [33]:
train_data = tuple([encoder_input_tr, decoder_input_tr, decoder_target_tr])

## Eval data

지금까지 진행한 것은 학습 데이터에 대한 전처리입니다.

그런데 말입니다, 과연 평가 데이터는 학습데이터랑 동일한 형태일까요?

사용하는 단어도 동일할까요?

게다가 실제 데이터라면 문장 속에 이메일이나, 전화번호등 다른 것들로 대체해야할 게 있지 않을까요?

참으로 고민할 게 많습니다. 이 모든 것들을 고려하면 좋겠지만, 현재 과제에서는 학습 데이터의 단어 집합에는 없는데 평가 데이터에서 등장하는 단어들은 어떻게 처리할지 한번 고민해봅시다.

**핵심 TODO**:만약에 본 적이 없는 token이 나온다면 어떻게 될까요?

--> Out of Vocabulary !  =지금까지 단어 집합에 없던 아이들!<br>

이를 한번에 처리해주는 토큰으로 \<UNK\>(=unknown token) 을 사용

EX)

train data : "나는 밥을 먹는다."<br>
\>> ["나", "밥", "먹다"]<br>
\>> [\<SOS\>, 나, 밥, 먹다, \<EOS\>] <BR>
\>> {\<SOS\>:0, \<EOS\>:1, \<UNK\>: 2, 나:3, 밥:4, 먹다:5}  <bR>
\>> [0, 3, 4, 5, 1] <BR>

val data: "나는 고기를 먹는다"<br>
\>> ['나', '고기' '먹다']<BR>
\>> [\<SOS\>, 나, \<UNK\>, 먹다, \<EOS\>] <BR>
\>> [0, 3, 2, 5, 1] <BR>

In [34]:
target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = filter(lambda x : x%2==0, target)
print(list(result))

[2, 4, 6, 8, 10]


In [35]:
src_seq_val = []
tar_seq_val = []


for raw_val_dat in tqdm(raw_val_data):
  # 문장별 tokenization
  src_tmp = CustomTokenizer(raw_val_dat['ko'])
  tar_tmp = CustomTokenizer(raw_val_dat['en'])

  # 입력 시퀀스에서 문장의 끝을 알리는 <EOS> 토큰 추가
  # 출력 시퀀스에서 시작과 끝을 알리는 <SOS>, <EOS> 토큰 추가
  src_tmp.append('<EOS>')
  tar_tmp.insert(0, '<SOS>')
  tar_tmp.append('<EOS>')

  # 처음보는 단어들은 일단 모르는 단어(OOV)로 표시해놓기  
  # 해당 과정에서는 지금 얻은 toekn들(src_tmp와 tar_tmp)들 중에서 학습 데이터의 단어 집합(src_vocab과 tar_vocab)에 없는 단어들을 찾아야 합니다.
  # 즉 위의 예시에서 "고기"에 해당하는 token을 찾아야 하고, 이를 list로 저장해봅시다.
  src_oov = list(filter(lambda x: x not in src_vocab, src_tmp))
  tar_oov = list(filter(lambda x: x not in tar_vocab, tar_tmp))

  # OOV들은 <UNK>로 바꾸는 과정입니다.
  # 지금 문장을 tokenization되어 있고, 앞서 구한 모르는 단어(OOV)들의 리스트(src_oov와 tar_oov)가 있는 상황에서
  # 해당 리스트에 속한 토큰들은 <UNK>토큰으로 바꿔주는 것을 구현하는 파트입니다.
  # 위의 예시에서 본다면 [<SOS>, 나, 고기, 먹다, <EOS>]를 [<SOS>, 나, <UNK>, 먹다, <EOS>]로 만들어주는 과정입니다.
  src_tmp = list(map(lambda x: '<UNK>' if x in src_oov else x, src_tmp))
  tar_tmp = list(map(lambda x: '<UNK>' if x in tar_oov else x, tar_tmp))
  
  # tokenization 끝난 문장 보관
  src_seq_val.append(src_tmp)
  tar_seq_val.append(tar_tmp)

100%|██████████| 1000/1000 [00:06<00:00, 149.80it/s]


In [36]:
tar_seq_val[5]

['<SOS>', '<UNK>', '<UNK>', '<UNK>', 'and', 'even', '<UNK>', 'skin', '<EOS>']

위의 샘플을 보면 생각보다 너무 많은 단어들이 UNK로 바뀌었습니다. 이게 과연 성능에 어떤 영향을 끼칠까요? 이를 방지할 수 있는 대책은 무엇일까요?

정답은 없습니다. 자유롭게 고민해보셔도 좋을 것 같습니다.

### UNK 토큰 추가 및 단어집합, token-index 업데이트

In [37]:
# UNK가 추가되면서 vocab과 index도 업데이트를 해야합니다.
# 즉 지금 방식으로 처리한다면, 지금 단어 집합에는 <UNK>라는 토큰이 없습니다.
# 그래서 UNK 토큰에도 정수를 할당해줘야 합니다.
# 지금 0은 padding에서 사용하고 있으니 아래의 과정은 마지막에 <UNK>를 추가해보는 것입니다.

In [38]:
# 입력 시퀀스의 마지막 토큰과 정수인코딩값 확인
list(src_to_index.items())[-1]

('힘쓰다', 5779)

In [39]:
# 현재 입력 시퀀스에 사용된 token들의 개수 확인
len(src_vocab)

5779

In [40]:
list(tar_to_index.items())[-1]

('zoom', 6910)

In [41]:
# 우리는 마지막 토큰 뒤에 UNK 토큰을 추가해봅시다
# 이 때 입력 시퀀스(src_to_index)와 출력시퀀스(tar_to_index)에 모두 사용해줘야 합니다.
src_to_index["<UNK>"] = 5780
tar_to_index["<UNK>"] = 6911

In [42]:
# 입력시퀀스의 단어집합 확인
print(list(tar_to_index.items())[:5])
print(list(tar_to_index.items())[-1:-5:-1])

[('!', 1), ('<EOS>', 2), ('<SOS>', 3), ('?', 4), ('A', 5)]
[('<UNK>', 6911), ('zoom', 6910), ('zones', 6909), ('zero', 6908)]


In [43]:
# 출력시퀀스의 단어집합 확인
print(list(src_to_index.items())[:5])
print(list(src_to_index.items())[-1:-5:-1])

[('!', 1), ('<EOS>', 2), ('?', 3), ('A', 4), ('AAA', 5)]
[('<UNK>', 5780), ('힘쓰다', 5779), ('힘들다', 5778), ('힘', 5777)]


### 정수 인코딩

In [44]:
# 평가 데이터에 대해서  정수 인코딩을 진행해주세요!
# 이 과정에서 중요한 점은 우린 학습데이터만 가지고 있으니 학습 데이터에서 사용했던 toekn-index로 인코딩을 진행해야 합니다!
# 참고 : 어떻게 하면 효율적 혹은 빠르게 할까요?

# encoder의 입력값 정수 인코딩 진행
encoder_input_val = list(map(lambda words: torch.Tensor(list(map(lambda c: src_to_index[c], words))).long(), src_seq_val))
# 우선 decoder에서 사용하는 문장에 대해 정수인코딩 진행
# decoder의 입력값은 마지막 <EOS>가 들어가면 안되니 마지막 토큰은 빼주기
decoder_input_val = list(map(lambda words: torch.Tensor(list(map(lambda c: tar_to_index[c], words))[:-1]).long(), tar_seq_val))
# 다만 decoder의 출력값은 <SOS>들어가면 안되니 첫번째 토큰은 빼주기
decoder_target_val = list(map(lambda words: torch.Tensor(list(map(lambda c: tar_to_index[c], words))[1:]).long(), tar_seq_val))


### Padding

In [45]:
# 주의사항: 여기에 사용된 값을은 정수일까요?
batch_first = True

encoder_input_val = pad_sequence(encoder_input_val, batch_first=batch_first).long()
decoder_input_val = pad_sequence(encoder_input_val, batch_first=batch_first).long()
decoder_target_val = pad_sequence(encoder_input_val, batch_first=batch_first).long()

In [46]:
val_data = tuple([encoder_input_val, decoder_input_val, decoder_target_val])

# Dataset

실습코드에서 사용한 것과 동일한 구조입니다! 

사실 앞서 처리가 진행되었던 모든 과정은 Dataset에서 __get_item__ 함수 내에서 처리해서 값을 리턴할 수도 있습니다.

다만 설명을 위해 이번 과제에서는 과정을 단계별로 설명 드렸습니다. 추후에 진행하실 때엔 __get_item__에서 구현해보셔도 좋을 것 같아요!

그리고 일부 단계의 경우 임의로 처리된 것들이 있습니다.(tokenization부터 stopwords 등등) 가령 이메일, 전화번호와 같은 단어들은 \<UNK\>처럼 새로운 토큰을 이용해서 처리할 수도 있으며 목적에 따라서 다른 tokenziation을 사용할 수 있습니다.

**(참고)**

 __get_item__말고 사전에 전처리를 다한 상태로 사용할 경우 사실 사용할 모든 데이터를 메모리에 올려놓고 사용하는 것과 같습니다.(변수에 할당되어 있으니)

그런데 우리가 데이터가 매매매매매우 많아진다면, 메모리가 부족해질 수도 있지 않을까요? 그렇기 때문에 때에 따라 data를 필요할 때 전처리를 하고 반환하기도 합니다. 여기서 data가 필요할때란 결국 __get_item__을 호출할 때가 될 것입니다.

정리하자면, 꼭 모든 데이터를 한번에 다 전처리해놓도 준비해놓지 않고도, 필요할 때 마다(get_item이 호출될 때마다) 전처리해서 결과를 전달하는 방법도 있습니다.

다만 해당 과제에서는 Seq2Seq에 익숙해지는 것을 목표로 하기에 이러한 점만 알고 가셔도 충분하기도 하고, 오히려 사전에 다 처리해놓는 현재 방식이 더 효율적일 수도 있습니다. 그렇기 때문에 이 점만 알고 가셔도 좋을 것 같습니다!

In [47]:
from torch.utils.data import Dataset, DataLoader

In [48]:
class textDataset(Dataset):
  def __init__(self, data, batch_first=True):
    super(textDataset, self).__init__()
    # tuple asgginment를 활용한다면 굳이 argument로 모든 데이터를 받지 않아도 됩니다!
    enc_inp, dec_inp, dec_out = data

    # 내부에서 사용할 변수들 정리하는 것으로, 모든 데이터가 torch.LongTensor로 정리된 값들이 저장됩니다.
    self.enc_inp = enc_inp
    self.batch_first= batch_first
    self.dec_inp = dec_inp
    self.dec_out = dec_out

  def __getitem__(self, idx):
     return self.enc_inp[idx], self.dec_inp[idx],self.dec_out[idx]

  def __len__(self):
    return self.enc_inp.size(int(~self.batch_first))

In [49]:
train_dataset = textDataset(train_data, batch_first=batch_first)
val_dataset = textDataset(val_data, batch_first=batch_first)

In [50]:
val_dataset[0]

(tensor([5780, 1729, 2012, 5780, 3757, 5416, 4342, 1370, 2775, 1886, 2791, 5174,
         1977, 2852, 3163,  751, 4003,    2,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([5780, 1729, 2012, 5780, 3757, 5416, 4342, 1370, 2775, 1886, 2791, 5174,
         1977, 2852, 3163,  751, 4003,    2,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([5780, 1729, 2012, 5780, 3757, 5416, 4342, 1370, 2775, 1886, 2791, 5174,
         1977, 2852, 3163,  751, 4003,    2,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]))

# Model

In [51]:
import torch.nn as nn
import torch.functional as F

## Encoder

본 모델에서는 LSTM를 기본 단위로 가지는 Seq2Seq 모델을 구성할 것

참고 [레퍼런스](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

In [52]:
class Encoder(nn.Module):
  def __init__(self, in_size, hid_size, tok_size, bat_first=True, num_lay=1, bidirect=False):
    super(Encoder, self).__init__()
    # 여기서 hidden_size는 LSTM의 hidden_size로 embedding vector를 몇차원의 벡터로 transform할 것인지 말합니다!
    self.hidden_size = hid_size
   
    # 앞서 설명한 내용대로 전체 사용하는 token의 수를 embedding_num에 해당하고, 이를 몇차원의 embedding dim으로 표현할 것인지 말합니다.
    # 이 때 LSTM의 관점에서 embedding dim은 input_size에 해당해서 개념적으로 embedding_dim=in_size입니다.
    self.embedding = nn.Embedding(tok_size, in_size)

    # layer를 구성할 때 LSTM 내부의 layer의 개수도 조정할 수 있습니다.
    self.num_layers = num_lay

    # 이것은 bidirectional 방식을 사용할 것인지를 얘기하는데,
    # 이는 현재 과정에서 고려하지 않는 값이지만 LSTM 레이어를 설정할 때 필요한 값이라 기록하긴 했습니다.
    self.num_directional = 2 if bidirect else 1

    self.batch_first = bat_first
    self.lstm = nn.LSTM(in_size, 
                        hid_size, 
                        num_layers = num_lay,
                        batch_first = bat_first,
                        bidirectional=bidirect)


  def forward(self, x, hidden):
    """
    x: (batch_size, seq_len)
    hidden: (batch_size, seq_len, hidden_size)
    """

    # emb : (batch_size, seq_len, input_size)
    emb = self.embedding(x)

    # out : (batch_size, seq_len, hidden_size)
    # hidden: (num_layers, batch_size, hidden_size)
    out, hidden = out, hidden = self.lstm(emb, hidden)
    return out, hidden

## Decoder

decoder의 hidden이 아닌 out으로 예측하고, 그 값 하나를 이용해서 다음 값 계산에 사용하기 때문에 seq_len =1이 된다.

In [53]:
class Decoder(nn.Module):
  def __init__(self, in_size, hid_size, tok_size, bat_first=True, num_lay=1, bidirect=False):
    super(Decoder, self).__init__()

    # 대부분의 방식은 사실 Encoder와 비슷하게 흘러갑니다.

    # LSTM이 embedding을 몇차원의 벡터로 변환할지 정하는 값입니다.
    self.hidden_size = hid_size

    # 여기서도 번역해야하는 언어의 token을 embedding 해야하고 이  때 embedding vector의 차원이 결국 LSTM입장에서의 Input size가 됩니다.
    self.embedding = nn.Embedding(tok_size, in_size)

    self.num_layers = num_lay
    self.num_directional = 2 if bidirect else 1
    self.batch_first = bat_first
    self.lstm = nn.LSTM(in_size, 
                        hid_size, 
                        num_layers = num_lay,
                        batch_first = bat_first,
                        bidirectional=bidirect)
    self.relu =  nn.ReLU()
    self.fc1 = nn.Linear(hidden_size, tok_size)

  def forward(self, x, hidden):
    """
    x: (batch_size, seq_len=1)
    hidden: (num_directional*num_layers, batch_size, hidden_size)
    """

    # emb: (batch_size, seq_len=1, input_size)
    emb = self.embedding(x[:, 0]).unsqueeze(1)

    # out : (batch_size, seq_len=1, hidden_size)
    # hidden: (num_layers, batch_size, hidden_size)
    out = self.relu(emb)
    out, hidden = self.lstm(out, hidden)

    # out: (batch_size, tok_size) : batch의 element별로 token에 대해 예측값을 반환
    out = self.fc1(out.squeeze(1))
    return out, hidden

In [54]:
def init_hidden(self, x):
  """
  레퍼런스 참고
  when batch_first=True (num_directional*num_layers, batch_size, hidden_size)
  """
  batch_size = x.size(0) if self.batch_first else x.size(1)
  h0 = torch.zeros(self.num_layers*self.num_directional, batch_size, self.hidden_size).to(device)
  c0 = torch.zeros(self.num_layers*self.num_directional, batch_size, self.hidden_size).to(device)
  return h0, c0

## Seq2Seq

In [55]:
class Seq2Seq(nn.Module):
  def __init__(self, enc, dec):
    super(Seq2Seq, self).__init__()
    assert enc.hidden_size == dec.hidden_size
    assert enc.num_layers == dec.num_layers

    self.encoder = enc
    self.decoder = dec

  def forward(self, enc_inp, dec_inp, use_teacher_force=True):
    """
    enc_inp : (batch_size, enc_seq_len)
    dec_inp : (batch_size, dec_seq_len)
    """

    # 어떤 값을 써야할까요?
    batch_size = enc_inp.size(0)
    enc_seq_len = enc_inp.size(1)
    dec_seq_len = dec_inp.size(1)
    # 최종 반환하는 것은 "토큰 별" score 값이니 이 점을 이용해서 token 개수 확인
    token_size =  self.decoder.fc1.out_features

    # decoder 확률(점수) 예측값 저장
    # outputs: (batch_size, seq_len, token_size)
    outputs = torch.zeros(batch_size, dec_seq_len, token_size)

    # Step 0: Encoder의 forward에 필요한 초기 hidden, cell state 계산
    enc_init_hidden = init_hidden(self.encoder, enc_inp)

    # Step 1: Encoder를 이용하여 context 벡터 생성
    # 참고: context 벡터는 output가 아님
    _, context = self.encoder(enc_inp, enc_init_hidden)

    # Step 2: Decoder의 초기 입력값을 먼저 할당
    # 참고: 첫 입력값으로 decoder의 입력값을 이용할 수 있음
    # 주의할 사항 
    #    - (batch_size, dec_seq_len)의 차원을 맞추기 
    #    - 이렇게 하는 이유는 decoder에서 forward에 들어가야하는 input의 구조(차원)이 정해져 있기 때문
    dec_inp_t = dec_inp[:, 0].unsqueeze(-1)

    # Step 3: Decoder를 이용하여 하나하나 계산(예측)하고, 입력값 업데이트하기
    # 주의사항
    #    - Encoder의 최종 리턴한 hidden state가 context 벡터고 이게 decoder에서 init_hidden에 해당
    #    - 앞서 첫 입력값은 직접 확인했으니 다음 입력값부터 하기에 range는 1부터 시작하는 구조로?!
    dec_hidden = context
    for t in range(1, dec_seq_len):
      out, dec_hidden = self.decoder(dec_inp_t, dec_hidden)
      outputs[:, t, :] = out

      if use_teacher_force:
        # 주의사항 : 차원 맞추기 
        # 해당 경우 decoder의 입력값을 가져다가 사용해야 합
        dec_inp_t = dec_inp[:, t].unsqueeze(-1)
      else:
        # 주의사항 : 차원 맞추기 
        # 해당 경우 이번 시점의 예측값을 다음 시점의 입력값으로 해야함
        dec_inp_t = out.argmax(1, keepdim=True)

    return outputs

# Setup

In [56]:
# padding에 해당하는 token도 고려하기
enc_token_size = len(src_to_index)+1
dec_token_size = len(tar_to_index)+1

# Hyper-parameter!!
# RAM 메모리 이슈로 다운 되는 경우 때문에 줄였습니다.
input_size = 4
hidden_size = 4

In [57]:
encoder = Encoder(input_size, hidden_size, enc_token_size)
decoder = Decoder(input_size, hidden_size, dec_token_size)

model = Seq2Seq(encoder, decoder).to(device)

## 생략된 내용들

모델 생성하는 과정에서 초기값이나 등등 부수적인 부분들은 현재 과제 코드상에서 생략되었습니다. 모델 개선에 관심있으신 분들은 수행하셔도 좋습니다.

In [58]:
# ~~~

# Train

In [59]:
def train(model, loader, optimizer, cri):
  model.train()
  loss_ep = 0

  for enc_input, dec_input, dec_target in loader:
    optimizer.zero_grad()

    # output: (batch_size, seq_len, token_size)
    output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=True)
    token_size = output.size(-1)

    # 지금 [문장, 문장, ...] 구조에서 문장들을 그냥 순서대로 이어 붙인 것
    # loss 계산을 위한 조정
    # output : (batch_size * seq_len, toekn_size)
    output = output.view(-1, token_size)

    # 위와 동일한 방식으로 label 값도 조정
    # dec_target: (batch_size, seq_len)
    # target (batch_size*seq_len,)
    target = dec_target.view(-1)
    
    loss = cri(output, target)

    loss.backward()
    optimizer.step()

    loss_ep += loss.item()
  return loss_ep/len(loader)

# Evaluation

In [60]:
import pandas as pd

In [61]:
def val(model, loader, cri):
  # 예측값도 확인하기 위해 예측값 저장하는 값
  result = torch.Tensor()

  model.eval()
  loss_ep = 0
  with torch.no_grad():
    for enc_input, dec_input, dec_target in loader:
      # 여기서 output은 en/decoder가 아닌 seq2seq의 결과값
      # output: (batch_size, seq_len, token_size)
      output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=False)

      # 평가할 땐 loss 말고 예측값도 반환하기 위해 저장
      # decode_idx = 해당 시점에서 예측한 token의 정수인코딩값. 
      # 주어진 확률(점수)값에서 최대값을 선택하기
      decode_idx = output.argmax(2).detach().cpu()
      result = torch.cat((result, decode_idx), dim=0)
  
      # loss 계산을 위한 조정
      # output : (batch_size * seq_len, toekn_size)
      token_size = output.size(-1)
      output = output.view(-1, token_size)

      # 위와 동일한 방식으로 label 값도 조정
      # dec_target: (batch_size, seq_len)
      # target (batch_size*seq_len,)
      target = dec_target.view(-1)

      loss = cri(output, target)
      loss_ep += loss.item()
  return loss_ep/len(loader), result

In [62]:
# 아래의 dictionary는 정수 인코딩 값을 넣으면 해당하는 원래 알파벳이 나옴
# padding에 사용한 0도 decode해줘야 하고 이 때 ''으로 decode해주기
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())
index_to_tar[0] = ''
index_to_src[0] = ''

In [63]:
def decode(model, sample):
  enc_input, dec_input, dec_output = sample
  enc_input = enc_input.unsqueeze(0)
  dec_input = dec_input.unsqueeze(0)
  
  model.eval()
  with torch.no_grad():
    output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=False)
    decode_idx = output.argmax(2)

  sentence_inp = pd.Series(enc_input.squeeze(0).detach().cpu().numpy()).apply(lambda x: index_to_src[x])
  sentence_out = pd.Series(decode_idx.squeeze(0).detach().cpu().numpy()).apply(lambda x: index_to_tar[x])
  sentence_act = pd.Series(dec_output.detach().cpu().numpy()).apply(lambda x: index_to_tar[x])

  print(f"입력 문장 : {' '.join(sentence_inp.tolist())}")
  print(f"실제 문장 : {' '.join(sentence_act.tolist())}")
  print(f"예측 문장 : {' '.join(sentence_out.tolist())}")

In [64]:
import time
from tqdm import tqdm
from torch import optim

In [65]:
# 학습은 아래의 코드를 이용하여 진행

train_lodaer = DataLoader(train_dataset, batch_size=1024)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()

train_loss = []
val_loss = []

epochs = 10
for epoch in tqdm(range(epochs)):
  start = time.time()

  train_loss_ep = train(model,train_lodaer,optimizer, criterion)
  val_loss_ep, val_token = val(model, val_loader, criterion)

  train_loss.append(train_loss_ep)
  val_loss.append(val_loss_ep)
  if epoch % 2 == 0:
    print(f"Epoch : {epoch}")
    print(f"Train Loss : {train_loss_ep:.4f}")
    print(f"Val Loss : {val_loss_ep:.4f}")
    decode(model, val_dataset[100])


 10%|█         | 1/10 [02:09<19:23, 129.24s/it]

Epoch : 0
Train Loss : 8.8570
Val Loss : 8.5490
입력 문장 : 저 <UNK> <UNK> 스마트 <UNK> <UNK> <UNK> 및 온도 센서 <UNK> 사용 무료 앱 제품 소개 하다 하다 <EOS>             
실제 문장 : meeting skyline skyline experts skyline skyline skyline compete hungry encryption skyline discussions close gears net engaging roles roles <EOS>             
예측 문장 :  know s can s can can s can can s can can s can can s can can s can can s can can s can can s can can s


 30%|███       | 3/10 [06:16<14:34, 124.88s/it]

Epoch : 2
Train Loss : 7.3479
Val Loss : 7.1401
입력 문장 : 저 <UNK> <UNK> 스마트 <UNK> <UNK> <UNK> 및 온도 센서 <UNK> 사용 무료 앱 제품 소개 하다 하다 <EOS>             
실제 문장 : meeting skyline skyline experts skyline skyline skyline compete hungry encryption skyline discussions close gears net engaging roles roles <EOS>             
예측 문장 :  s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s s


 50%|█████     | 5/10 [10:23<10:19, 123.92s/it]

...

 70%|███████   | 7/10 [14:28<06:09, 123.25s/it]

Epoch : 6
Train Loss : 3.6938
Val Loss : 4.2912
입력 문장 : 저 <UNK> <UNK> 스마트 <UNK> <UNK> <UNK> 및 온도 센서 <UNK> 사용 무료 앱 제품 소개 하다 하다 <EOS>             
실제 문장 : meeting skyline skyline experts skyline skyline skyline compete hungry encryption skyline discussions close gears net engaging roles roles <EOS>             
예측 문장 :                                


 90%|█████████ | 9/10 [18:34<02:03, 123.21s/it]

Epoch : 8
Train Loss : 2.8305
Val Loss : 3.7198
입력 문장 : 저 <UNK> <UNK> 스마트 <UNK> <UNK> <UNK> 및 온도 센서 <UNK> 사용 무료 앱 제품 소개 하다 하다 <EOS>             
실제 문장 : meeting skyline skyline experts skyline skyline skyline compete hungry encryption skyline discussions close gears net engaging roles roles <EOS>             
예측 문장 :                                


100%|██████████| 10/10 [20:38<00:00, 123.86s/it]


In [66]:
# 학습을 진행한 다음, 모델 백업해놓기
torch.save(model.state_dict(), 'model_hw.pt')